# 一次性训练（低级接口）示例

### 初始化SDK与Anylearn后端引擎连接

In [1]:
from anylearn.config import init_sdk
init_sdk('http://192.168.10.22:31888', 'xlearn', '123456')

### 创建数据集和算法

In [2]:
from anylearn.interfaces.mirror import Mirror
from anylearn.interfaces.resource.dataset import Dataset
from anylearn.interfaces.resource.algorithm import Algorithm

dset = Dataset(name="TY20T-SDK", description="天远20吨保养数据集", filename="ty20t.zip", is_zipfile=True)
dset.save()
print(f"数据集：{dset.id}")

mirror = next(m for m in Mirror.get_list() if m.name == "pytorch1.5")
algo = Algorithm(name="SSD-SDK", filename="SSD-22-clean.zip", is_zipfile=True, mirror_id=mirror.id,
                 train_params="[{\"name\":\"dataset\",\"type\":\"dataset\",\"suggest\":1},{\"name\":\"max_iter\",\"type\":\"int\",\"suggest\":1,\"default\":\"12000\",\"scope\":\"<-Infinity,Infinity>\"}]",
                 evaluate_params="[{\"name\":\"dataset\",\"type\":\"dataset\",\"suggest\":1},{\"name\":\"model_path\",\"alias\":\"\",\"description\":\"\",\"type\":\"model\",\"suggest\":1}]")
algo.save()
print(f"算法：{algo.id}")

数据集：DSET1562886711ebbdad2e92612489ab
算法：ALGO21d4886711ebbdad2e92612489ab


### 上传本地数据集和算法文件

In [3]:
from threading import Thread
import time
from anylearn.interfaces.resource.resource_uploader import AsyncResourceUploader
from anylearn.interfaces.resource.resource import Resource, ResourceState

def __on_progress(curr, total):
    prog = int(100 * curr / total)
    if not (prog % 20):
        print(f"Uploaded {prog}%")

uploader = AsyncResourceUploader(on_progress=__on_progress)

print("数据集上传中：")
t_dset = Thread(target=Resource.upload_file,
                kwargs={
                    'resource_id': dset.id,
                    'file_path': "/Users/yhuang/dev/anylearn-resources/datasets/ty20t.zip",
                    'uploader': uploader,
                })
t_dset.start()
t_dset.join()
while dset.state != ResourceState.READY:
    time.sleep(5)
    dset.get_detail()
print("数据集上传成功且已就绪")

print("算法上传中：")
t_algo = Thread(target=Resource.upload_file,
                kwargs={
                    'resource_id': algo.id,
                    'file_path': "/Users/yhuang/dev/anylearn-resources/algo/SSD-22-clean.zip",
                    'uploader': uploader,
                })
t_algo.start()
t_algo.join()
while algo.state != ResourceState.READY:
    time.sleep(5)
    algo.get_detail()
print("算法上传成功且已就绪")

数据集上传中：
Uploaded 20%
Uploaded 40%
Uploaded 60%
Uploaded 80%
Uploaded 100%
数据集上传成功且已就绪
算法上传中：
Uploaded 60%
Uploaded 100%
算法上传成功且已就绪


### 创建项目

In [4]:
from anylearn.interfaces.project import Project
project = Project(name="Quick-train-SSD-TY20T", datasets=[dset.id])
project.save()
project.get_detail()
project

Project(id='PROJc0ec886711ebbdad2e92612489ab', name='Quick-train-SSD-TY20T', description='', visibility=3, create_time='2021-03-19 11:59:56', update_time='2021-03-19 11:59:56', creator_id='USERfb6c6d2111eaadda13fd17feeac7', datasets=['DSET1562886711ebbdad2e92612489ab'], owner=['USERfb6c6d2111eaadda13fd17feeac7'])

### 创建训练任务

In [5]:
from anylearn.interfaces.train_task import TrainTask
train_task = TrainTask(name="SSD-TY20T", project_id=project.id,
                       algorithm_id=algo.id,
                       files=[dset.id],
                       train_params=f"{{\"dataset\":\"${dset.id}\",\"max_iter\":\"100\"}}")
train_task.save()
train_task.get_detail()
train_task

TrainTask(name='SSD-TY20T', description='', state=0, visibility=3, creator_id='USERfb6c6d2111eaadda13fd17feeac7', owner=['USERfb6c6d2111eaadda13fd17feeac7'], project_id=None, algorithm_id='ALGO21d4886711ebbdad2e92612489ab', train_params='{"dataset":"$DSET1562886711ebbdad2e92612489ab","max_iter":"100"}', files='DSET1562886711ebbdad2e92612489ab', results_id='FILE7b0c886711ebbdad2e92612489ab', secret_key='TKEY2074886711ebbdad2e92612489ab', create_time='2021-03-19 11:59:56', finish_time='', envs='', gpu_num=1, hpo=False, hpo_search_space=None, id='TRAIb5ba886711ebbdad2e92612489ab')

### 追踪训练进度

In [6]:
import time

status = train_task.get_status()
while 'state' not in status:
    print("Waiting...")
    time.sleep(10)
    status = train_task.get_status()
while status['state'] not in ["success", "fail"]:
    print(f"Progress: {int(100 * float(status['process']))}%")
    time.sleep(30)
    status = train_task.get_status()
status['state']

Waiting...
Progress: 0%
Progress: 18%
Progress: 50%
Progress: 82%


'success'